In [47]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [48]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

* Calculate the total budget
* Calculate the total number of schools
* Calculate the total number of students

In [49]:
schools_list = school_data_complete.school_name.unique()
schools_number =len(schools_list)
tot_stud_1 = len(school_data_complete.student_name.unique())
tot_stud_2 = school_data_complete.school_name.value_counts()
tot_stud = tot_stud_2.sum()
tot_budget = school_data_complete.budget.unique().sum()
print(tot_stud_2)
border = "#"*77
print("\n{}\nTotal budget: ${:,}\nTotal number of schools:{}".format(border, tot_budget, schools_number))
print("Total number of students: by school records - {:,}, by unique name - {:,}\n{}".format(tot_stud, tot_stud_1, border))

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

#############################################################################
Total budget: $24,649,428
Total number of schools:15
Total number of students: by school records - 39,170, by unique name - 32,715
#############################################################################


* Calculate the average math score
* Calculate the average reading score
* Calculate the percentage of students with a passing math score (70 or greater)
* Calculate the percentage of students with a passing reading score (70 or greater)
* Calculate the percentage of students who passed math **and** reading (% Overall Passing)
* Create a dataframe to hold the above results
* Optional: give the displayed data cleaner formatting

In [60]:
avg_math = school_data_complete.math_score.sum()/tot_stud
avg_read = school_data_complete.reading_score.sum()/tot_stud
st_pass_math = len(school_data_complete.math_score[school_data_complete.math_score>=70])
st_pass_read = len(school_data_complete.reading_score[school_data_complete.reading_score>=70])
st_pass_both = len(school_data_complete[(school_data_complete.math_score>=70)
                                                    & (school_data_complete.reading_score>=70)])
st_pass_one = len(school_data_complete.loc[(school_data_complete.math_score>=70)
                                                    | (school_data_complete.reading_score>=70)])

data_total = ["Total", schools_number, tot_stud, tot_budget, tot_budget/tot_stud, avg_math, avg_read,
            st_pass_math/tot_stud*100, st_pass_read/tot_stud*100, st_pass_both/tot_stud*100, st_pass_one/tot_stud*100]
data_col = ["School Name", "Number of Schools/School Type", "Total Students", "Total School Budget",
            "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math",
            "% Passing Reading", "%Overall Passing (Math and Reading)", "%At Least One Test Passing (Math or Reading)"]
total_df = pd.DataFrame(columns = data_col)
total_df.loc[0] = data_total
print (type(total_df))
# pd.set_option('display.max_colwidth', 10)
# pd.options.display.float_format = '{:,.2f}'.format
print("\nDistrict Summary:\n{}\n{}\n{}".format(border, total_df, border))

<class 'pandas.core.frame.DataFrame'>

District Summary:
#############################################################################
  School Name Number of Schools/School Type Total Students  \
0      Total          15                         39170       

  Total School Budget  Per Student Budget  Average Math Score  \
0   24649428              629.29               78.99            

   Average Reading Score  % Passing Math  % Passing Reading  \
0      81.88                  74.98           85.81           

   %Overall Passing (Math and Reading)  \
0      65.17                             

   %At Least One Test Passing (Math or Reading)  
0      95.61                                     
#############################################################################


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [35]:
schools = pd.Series([schools_list])
print(schools)

0    [Huang High School, Figueroa High School, Shel...
dtype: object


In [36]:
for i in range(len(schools_list)):
    total_df.append["School Name": schools,[]]
print(total_df)

NameError: name 'schools' is not defined

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type